In [1]:
import gzip
import torch
import pickle
import numpy as np
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor, Lambda
from torch.utils.data import DataLoader

In [3]:
def MNIST_loaders(train_batch_size=1, test_batch_size=1):

    transform = Compose([
        ToTensor(),
        Lambda(lambda x: torch.flatten(x))])

    train_loader = DataLoader(
        MNIST('./data/', train=True,
              download=True,
              transform=transform),
        batch_size=train_batch_size, shuffle=False)

    test_loader = DataLoader(
        MNIST('./data/', train=False,
              download=True,
              transform=transform),
        batch_size=test_batch_size, shuffle=False)

    return train_loader, test_loader

In [ ]:
train_loader, test_loader = MNIST_loaders()

In [5]:
lbl_true=0
count=0
k=6

for test in test_loader:
  original_label = int(test[1])
  test_vect = test[0].squeeze(0)
  byte_vect = pickle.dumps(test_vect)
  cx1=len(gzip.compress(byte_vect))

  labels = []
  distances = []

  for train in train_loader:
    lbl = int(train[1])
    train_vect = train[0].squeeze(0)
    byte_vect = pickle.dumps(train_vect)
    cx2=len(gzip.compress(byte_vect))

    conc_vect = torch.cat((train_vect, test_vect))
    byte_vect = pickle.dumps(conc_vect)

    cx1x2 = len(gzip.compress(byte_vect))

    dist = (cx1x2 - min(cx1, cx2))/max(cx1, cx2)
    labels.append(lbl)
    distances.append(dist)

  sorted_idx=np.argsort(np.array(distances))
  labels = np.array(labels)
  top_k_class=labels[sorted_idx[:k]]
  lbl, counter = np.unique(top_k_class, return_counts=True)
  final_label = lbl[np.argsort(-counter)[0]]

  if final_label == original_label:
    lbl_true +=1
  count+=1
  if count ==10:
    break


print(lbl_true/count)

0.4
